In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
df = pd.read_csv('sample_data/jamb_exam_results.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_')
df = df.drop(columns=['student_id'])
df = df.fillna(0)
train_data, temp_data = train_test_split(df, test_size=0.4, random_state=1)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=1)

# Вопрос 1: Какой признак используется для разбиения данных?

In [4]:
train_dicts = train_data.drop(columns=['jamb_score']).to_dict(orient='records')
val_dicts = val_data.drop(columns=['jamb_score']).to_dict(orient='records')
test_dicts = test_data.drop(columns=['jamb_score']).to_dict(orient='records')

dv = DictVectorizer(sparse=True)

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

y_train = train_data['jamb_score'].values
y_val = val_data['jamb_score'].values
y_test = test_data['jamb_score'].values
model = DecisionTreeRegressor(max_depth=1, random_state=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f"RMSE на валидационной выборке: {rmse}")

RMSE на валидационной выборке: 44.85913145740971


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ответ: Study_hours_per_week

# Вопрос 2: Какое значение RMSE у этой модели на валидационных данных?

In [5]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_val)
rmse_rf = mean_squared_error(y_val, y_pred_rf, squared=False)

print(f"RMSE случайного леса на валидационных данных: {rmse_rf:.2f}")

RMSE случайного леса на валидационных данных: 43.16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Ближайший ответ: 42.13

# Вопрос 3: После какого значения n_estimators RMSE перестает улучшаться?
# Вопрос 4: Какое значение max_depth оказалось лучшим по среднему RMSE?

In [6]:
import numpy as np

rmse_values = []
n_estimators_range = range(10, 201, 10)

for n in n_estimators_range:
    rf_model = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred_rf, squared=False)
    rmse_values.append(rmse)

rmse_values = np.array(rmse_values)
optimal_n_estimators = n_estimators_range[np.argmin(rmse_values)]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

In [8]:
print(f"n_estimators: {optimal_n_estimators}")

n_estimators: 180


Ближе к ответу 200, по среднему лучшим было значение 10

# Вопрос 5. Какой признак оказался самым важным (из этих четырех)?

In [9]:
rf_model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

importances = rf_model.feature_importances_
feature_names = dv.get_feature_names_out()

important_features = sorted(zip(importances, feature_names), reverse=True)
most_important_feature = important_features[0]

print(f"Самый важный признак: {most_important_feature[1]}, Важность: {most_important_feature[0]:.3f}")

Самый важный признак: study_hours_per_week, Важность: 0.254


# Вопрос 6: Какое значение eta приводит к лучшему значению RMSE на валидационном наборе данных?

In [11]:
import xgboost as xgb
from xgboost import DMatrix

dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

watchlist = [(dtrain, 'train'), (dval, 'validation')]

def train_xgb(eta):
    xgb_params = {
        'eta': eta,
        'max_depth': 6,
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'nthread': 8,
        'seed': 1,
        'verbosity': 1,
    }

    model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)
    return model

model_eta_03 = train_xgb(eta=0.3)
model_eta_01 = train_xgb(eta=0.1)

rmse_eta_03 = model_eta_03.best_score
rmse_eta_01 = model_eta_01.best_score

print(f"RMSE при eta=0.3: {rmse_eta_03:.3f}")
print(f"RMSE при eta=0.1: {rmse_eta_01:.3f}")

if rmse_eta_01 < rmse_eta_03:
    print("Лучшее: 0.1")
elif rmse_eta_01 > rmse_eta_03:
    print("Лучшее: 0.3")
else:
    print("Одинаково")

RMSE при eta=0.3: 40.688
RMSE при eta=0.1: 40.166
Лучшее: 0.1


1. Study_hours_per_week
2. 42.13
3. 200
4. 10
5. study_hours_per_week
6. 0,1